# Load epub book

In [88]:
# Import libraries
import os
from langchain_community.document_loaders import UnstructuredEPubLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

import chromadb
from uuid import uuid4
from chromadb.utils import embedding_functions

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig

In [2]:
# TODO: Load document
chunk_size = 1024
chunk_overlap = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

epub_loader = UnstructuredEPubLoader(file_path='/content/docs/charles-dickens_a-christmas-carol.epub')
chunks = epub_loader.load_and_split(text_splitter)


[WARNING] Sandbox argument was used, but pandoc version is too low. Ignoring argument.


In [10]:
# TODO Split document
print(len(chunks))
print(chunks[100])
print(chunks[100].page_content)
print(chunks[100].metadata['source'])

203
page_content='For the people who were shovelling away on the housetops were jovial and full of glee; calling out to one another from the parapets, and now and then exchanging a facetious snowball﻿—better-natured missile far than many a wordy jest﻿—laughing heartily if it went right, and not less heartily if it went wrong. The poulterers’ shops were still half open, and the fruiterers’ were radiant in their glory. There were great, round, potbellied baskets of chestnuts, shaped like the waistcoats of jolly old gentlemen, lolling at the doors, and tumbling out into the street in their apoplectic opulence: There were ruddy, brown-faced, broad-girthed Spanish onions, shining in the fatness of their growth like Spanish friars, and winking from their shelves in wanton slyness at the girls as they went by, and glanced demurely at the hung-up mistletoe. There were pears and apples clustered high in blooming pyramids; there were bunches of grapes, made, in the shopkeepers’ benevolence, to d

In [ ]:
# TODO Examine chunk


# Create embeddings

In [3]:
# TODO: Create embedding model
embed_model_name = "BAAI/bge-small-en-v1.5"
#embed_model_name = "all-MiniLM-L6-v2"

embed_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=embed_model_name)


model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: BAAI/bge-small-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# TODO: Explore embedding model
text = 'hello, world'
text = chunks[100].page_content
emb_text = embed_func([ text ])
print(emb_text[0].shape)
print(emb_text[0])

(384,)
[-3.74143459e-02 -1.72982123e-02  3.23804170e-02  3.15706730e-02
  3.94253954e-02 -3.27906124e-02  7.73764774e-02 -2.20255973e-03
 -7.40020648e-02 -2.07088590e-02  3.57349999e-02 -1.13295075e-02
  2.27336176e-02 -1.78614650e-02  3.54938358e-02  5.41082285e-02
  1.57259349e-02  1.56856310e-02 -8.48050416e-02  8.08566518e-04
 -6.30097762e-02 -6.33994192e-02  1.18485130e-02  5.39479218e-02
  1.35819837e-02  3.28279361e-02  1.74964443e-04  1.03100603e-02
 -2.75508258e-02 -3.98066714e-02 -2.81423926e-02  6.43461570e-02
  3.91568504e-02 -8.37504771e-03 -1.07788909e-02  3.24381664e-02
  9.43938643e-02 -3.07309870e-02 -3.18103544e-02  6.05167961e-03
  7.67107755e-02 -5.00436425e-02 -3.72683480e-02  1.10833319e-02
 -1.53249390e-02  1.79562680e-02 -4.23986465e-02  1.67341065e-02
 -2.34105811e-02  2.12488510e-02 -2.97802854e-02 -2.16397010e-02
 -4.93492074e-02 -1.27006331e-02  6.80123866e-02  1.99782029e-02
 -1.03280656e-02 -2.54409742e-02  3.40981260e-02  1.92671958e-02
 -6.06929604e-03 -

In [29]:
# TODO: Prepare the chunks for inserting into Chroma
texts = [ d.page_content for d in chunks ]

text_ids = [ str(uuid4())[:8] for _ in range(len(texts)) ]

text_metadata = [ { 'source': d.metadata['source'], 'len': len(d.page_content) } for d in chunks ]

print(len(text_metadata))
print(text_metadata[0])


203
{'source': '/content/docs/charles-dickens_a-christmas-carol.epub', 'len': 919}


In [28]:
for i in chunks:
  print(len(i.page_content))

919
963
890
913
677
1002
206
875
845
744
951
1019
950
1020
999
839
984
1023
627
982
975
921
785
922
710
865
754
977
889
473
1019
996
636
928
669
981
928
664
907
906
982
708
424
851
872
577
944
980
720
1016
160
976
1015
988
853
597
617
850
776
860
989
934
975
255
1021
170
957
862
687
443
1015
700
728
762
1006
937
983
873
1015
775
628
1019
781
1016
255
951
998
300
808
632
1022
233
112
1020
185
327
983
517
672
949
1015
782
1022
493
899
954
606
441
612
1023
92
855
915
636
522
1023
173
978
980
828
893
904
788
1020
282
515
892
734
733
703
750
474
775
811
737
992
850
625
1010
933
1020
775
930
283
1010
782
843
1011
894
945
996
731
913
932
723
1004
255
880
722
469
753
745
933
958
1006
954
967
912
748
988
984
922
1018
729
902
843
891
545
689
894
1021
965
915
1008
821
919
960
946
847
904
800
805
998
985
836
894
914
460
771
924
667
904
1000


In [30]:
# TODO: Create ephemeral Chroma client and save chunks
# create a chroma client
ch_client = chromadb.Client()

col_name = 'carol'


In [31]:
# delete old chunks before saving
try:
    ch_client.delete_collection(name=col_name)
except:
    pass


In [32]:
# create the collection
carol_col = ch_client.create_collection(
    name=col_name,
    embedding_function=embed_func
)

In [33]:
# Insert the docs into the database with metadatas
carol_col.add(
    documents=texts,
    metadatas=text_metadata,
    ids=text_ids
)

In [34]:
# TODO: Print number of documents in collection
print(carol_col.count())

203


In [110]:
# TODO: Query collection
query = 'Who is Marley?'

results = carol_col.query(
    query_texts=query,
    # filter by metadata
    where={ "len": { "$gt": 750 }},
    n_results=5
)

print(results)

{'ids': [['f3e07b3e', '37a98909', '84feddc3', '5d447da0', '65ffe80b']], 'embeddings': None, 'documents': [['“How now!” said Scrooge, caustic and cold as ever. “What do you want with me?”\n\n“Much!”\ufeff—Marley’s voice; no doubt about it.\n\n“Who are you?”\n\n“Ask me who I was.”\n\n“Who were you, then?” said Scrooge, raising his voice. “You’re particular, for a shade.” He was going to say “to a shade,” but substituted this, as more appropriate.\n\n“In life I was your partner, Jacob Marley.”\n\n“Can you\ufeff—can you sit down?” asked Scrooge, looking doubtfully at him.\n\n“I can.”\n\n“Do it, then.”\n\nScrooge asked the question, because he didn’t know whether a ghost so transparent might find himself in a condition to take a chair; and felt that in the event of its being impossible, it might involve the necessity of an embarrassing explanation. But the Ghost sat down on the opposite side of the fireplace, as if he were quite used to it.\n\n“You don’t believe in me,” observed the Ghost.\

In [41]:
for i in results['ids'][0]:
  r = carol_col.get(i)
  print(r)

{'ids': ['f3e07b3e'], 'embeddings': None, 'documents': ['“How now!” said Scrooge, caustic and cold as ever. “What do you want with me?”\n\n“Much!”\ufeff—Marley’s voice; no doubt about it.\n\n“Who are you?”\n\n“Ask me who I was.”\n\n“Who were you, then?” said Scrooge, raising his voice. “You’re particular, for a shade.” He was going to say “to a shade,” but substituted this, as more appropriate.\n\n“In life I was your partner, Jacob Marley.”\n\n“Can you\ufeff—can you sit down?” asked Scrooge, looking doubtfully at him.\n\n“I can.”\n\n“Do it, then.”\n\nScrooge asked the question, because he didn’t know whether a ghost so transparent might find himself in a condition to take a chair; and felt that in the event of its being impossible, it might involve the necessity of an embarrassing explanation. But the Ghost sat down on the opposite side of the fireplace, as if he were quite used to it.\n\n“You don’t believe in me,” observed the Ghost.\n\n“I don’t,” said Scrooge.\n\n“What evidence would

# Question and Answer LLM
In this exercise you will implement a question and answer LLM for the 'A Christmas Carol' book that you have chunked and saved.

The workflow is as follows:
1. Assume you ask the following question regarding the book eg. `"Who is Scrooge?"`?
2. Query the relevant context from Chroma with the question or facts from the question.
3. Combine the question and the top 5 context return by Chroma into a prompt
4. Use `google/flan-t5-base` to answer the question.

Look through the FLAN templates in [Github](https://github.com/google-research/FLAN/blob/main/flan/templates.py) and select an appropriate template for this workshop.

Do not worry about the accuracy of the result. Focus on implementing the solution. We will discuss the nuances of the solution at the end of the workshop.

Use your RAG workflow to answer the provided questions in `questions_for_rag.txt` file.

In [50]:
import sys

In [106]:
# TODO Your code
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config = GenerationConfig(
    do_sample=True,
    temperature=0.7,
    top_p=0.95
)

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [122]:
# TODO Your code
question = "What is the name of Scrooge's underpaid clerk?"
question = "What is the name of Bob Cratchit's youngest son who is ill?"
#question = "What does Scrooge do on Christmas morning after his transformation?"
#question = "What was Scrooge's first name as a boy, revealed by the Ghost of Christmas Past?"
#question = "What specific, generous act does Scrooge perform for the Cratchit family on Christmas morning?"

prompt_0 = f"{question}\n\nWhat is sentence that verbalizes this data?"
#prompt_0 = f"What sentence would provide a factual answer to this question: \"{question}\""

enc_prompt_0 = tokenizer(prompt_0, return_tensors='pt')
enc_result = model.generate(enc_prompt_0['input_ids'], generation_config=config)

result = tokenizer.decode(enc_result[0], skip_special_tokens=True)
print(result)

Bob Cratchit's youngest son is ill.


In [123]:
db_result = carol_col.query(
    query_texts=result,
    n_results=5
)

result_ids = db_result['ids'][0]
#ids = [ result_ids[i] for i in range(len(result_ids)) if db_result['distances'][0][i] <= .3 ]
ids = [ result_ids[i] for i in range(len(result_ids)) ]

#if (len(ids) <= 0):
#  raise Exception('I do not know how to answer the question')

context = ""
for i in ids:
  chunk = carol_col.get(i)
  context += chunk['documents'][0] + "\n"

print(context)

She hurried out to meet him; and little Bob in his comforter﻿—he had need of it, poor fellow﻿—came in. His tea was ready for him on the hob, and they all tried who should help him to it most. Then the two young Cratchits got upon his knees, and laid, each child, a little cheek against his face, as if they said, “Don’t mind it, father. Don’t be grieved!”

Bob was very cheerful with them, and spoke pleasantly to all the family. He looked at the work upon the table, and praised the industry and speed of Mrs. Cratchit and the girls. They would be done long before Sunday, he said.

“Sunday! You went today, then, Robert?” said his wife.

“Yes, my dear,” returned Bob. “I wish you could have gone. It would have done you good to see how green a place it is. But you’ll see it often. I promised him that I would walk there on a Sunday. My little, little child!” cried Bob. “My little child!”
So Martha hid herself, and in came little Bob, the father, with at least three feet of comforter, exclusive 

In [124]:
# TODO Your code
prompt_1 = f"{context}\n\nAnswer this question based on the article: {question}"


enc_prompt_1 = tokenizer(prompt_1, return_tensors='pt')
enc_result = model.generate(enc_prompt_1['input_ids'])

result = tokenizer.decode(enc_result[0], skip_special_tokens=True)
print(f'\n\nQuestion: {question}')
print(f'\nAnswer: {result}')



Question: What is the name of Bob Cratchit's youngest son who is ill?

Answer: Tiny Tim


# Discussion

1. How did your solution perform?
2. Where do you think are the issues?
3. How can you improve it?